In [1]:
!pip install transformers[torch]
!pip install sentencepiece
!pip install datasets
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 4.2 MB/s eta 0:00:00


In [2]:
import random
from tabulate import tabulate
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.metrics import classification_report

# datasets
from datasets import Dataset
from datasets import DatasetDict
from datasets import load_metric
from datasets import load_dataset

# transformers
from transformers import Trainer
from transformers import AutoTokenizer
from transformers import DataCollatorWithPadding
from transformers import TrainingArguments
from transformers import AutoModelForSequenceClassification
from transformers import IntervalStrategy

import torch
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset

from sklearn.metrics import accuracy_score, f1_score

import evaluate

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [3]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [4]:
from datasets.utils.logging import disable_progress_bar
from transformers import logging


disable_progress_bar()
logging.set_verbosity_error()

In [5]:
directory = "/kaggle/input/amazon-with-translated"
# directory = "/content/drive/MyDrive/MSc/NLP/nlp-project"

In [6]:
train_path = f"{directory}/train.csv"
test_path = f"{directory}/test.csv"
valid_path = f"{directory}/valid.csv"
translated_test_path = f"{directory}/amazon_translated_body_and_title_with_originals_all_stars.csv"

In [7]:
train_df = pd.read_csv(train_path)
test_df = pd.read_csv(test_path)
valid_df = pd.read_csv(valid_path)
translated_test_df = pd.read_csv(translated_test_path)

In [8]:
translated_test_df.rename(columns={"review_body": "review_body_original", "review_title": "review_title_original"}, inplace=True)

In [9]:
translated_test_df.rename(columns={"translated_body": "review_body", "translated_title": "review_title"}, inplace=True)

In [10]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(device)


cuda


In [11]:
SEED = 111

# Set the random seed for Python to SEED
random.seed(SEED)

# Set the random seed for numpy to SEED
np.random.seed(SEED)

# Set the random seed for torch to SEED
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

In [12]:
def verbose_print(msg, verbose=False):
    """
    Print the message if verbose is True

    Args:
        msg (str): The message to print
        verbose (bool, optional): Whether to print the message or not. Defaults to False.

    Returns:
        None
    """

    if verbose:
        print(msg)


def drop_data(df, stars, languages, columns_to_drop):
    """
    Drop rows that have stars or languages that are not in the list

    Args:
        df (pandas.DataFrame): The dataframe to drop rows from
        stars (list): The list of stars to keep
        languages (list): The list of languages to keep
        columns_to_drop (list): The list of columns to drop

        Returns:
            pandas.DataFrame: The dataframe with the dropped rows
    """

    df = df[df["stars"].isin(stars)]
    df = df[df["language"].isin(languages)]
    if len(columns_to_drop) > 0:
        df.drop(columns=columns_to_drop, inplace=True)
    df.reset_index(drop=True, inplace=True)

    return df


def prepare_data(
    train, test, valid, columns_to_drop, stars=[1, 2, 3, 4], languages=["en"]
):
    """
    Prepare the data by dropping rows that have stars or languages that are not in the list

    Args:
        train (pandas.DataFrame): The training dataframe
        test (pandas.DataFrame): The test dataframe
        valid (pandas.DataFrame): The validation dataframe
        columns_to_drop (list): The list of columns to drop
        stars (list, optional): The list of stars to keep. Defaults to [1, 2, 3, 4].
        languages (list, optional): The list of languages to keep. Defaults to ["en"].

    Returns:
        pandas.DataFrame, pandas.DataFrame, pandas.DataFrame: The prepared dataframes
    """

    train = drop_data(train, stars, languages, columns_to_drop)
    test = drop_data(test, stars, languages, columns_to_drop)
    valid = drop_data(valid, stars, languages, columns_to_drop)

    return train, test, valid


def reduce_dataset(df, stars, languages, num_of_rows_to_drop, verbose=False):
    """
    Reduce the number of rows in the dataset randomly choosing num_of_rows_to_drop rows to drop

    Args:

        df (pandas.DataFrame): The dataframe to reduce
        stars (list): The list of stars to keep
        languages (list): The list of languages to keep
        num_of_rows_to_drop (int): The number of rows to drop
        verbose (bool, optional): Whether to print the number of rows before and after the reduction. Defaults to False.

        Returns:
            pandas.DataFrame: The dataframe with the reduced number of rows
    """

    for lang in languages:
        for star in stars:
            verbose_print(f"Language: {lang}, Stars: {star}", verbose)
            verbose_print(
                f'Number of rows before: {len(df[(df["language"] == lang) & (df["stars"] == star)])}',
                verbose,
            )
            random_indices = np.random.choice(
                df[(df["language"] == lang) & (df["stars"] == star)].index,
                num_of_rows_to_drop,
                replace=False,
            )
            df.drop(index=random_indices, inplace=True)
            verbose_print(
                f'Number of rows after: {len(df[(df["language"] == lang) & (df["stars"] == star)])}',
                verbose,
            )

    return df


def replace_mapping(df, label, mapping):
    """
    Replace labels with the ones in the mapping

    Args:

        df (pandas.DataFrame): The dataframe to replace the labels in
        label (str): The label to replace
        mapping (dict): The mapping to use for the replacement

        Returns:
            pandas.DataFrame: The dataframe with the replaced labels
    """

    for k, v in mapping.items():
        df[label].replace(k, v, inplace=True)
    return df


def prepare_truncation(data_df, tokenizer, m, n):
    """
    Truncate the reviews to m+n tokens

    Args:

        data_df (pandas.DataFrame): The dataframe to truncate
        tokenizer (transformers.tokenization_utils_base.PreTrainedTokenizerBase): The tokenizer to use
        m (int): The number of tokens to keep from the beginning of the review
        n (int): The number of tokens to keep from the end of the review

        Returns:
            pandas.DataFrame: The dataframe with the truncated reviews
    """
    for i, r in tqdm(data_df.iterrows(), total=len(data_df), desc="Processing reviews"):
        tokenized_row = tokenizer.tokenize(r["review_body"])
        if len(tokenized_row) > m + n:
            data_df.loc[i, "review_body"] = tokenizer.convert_tokens_to_string(
                tokenized_row[:m] + tokenized_row[-n:]
            )

    return data_df


def print_using_tabulate(data):
    """
    Print the classification report using tabulate

    Args:

        data (dict): The classification report to print

        Returns:
            None
    """
    table_data = []
    for key, values in data.items():
        if key != "macro avg" and key != "weighted avg":
            if isinstance(values, dict):
                row = [
                    key,
                    values["precision"],
                    values["recall"],
                    values["f1-score"],
                    values["support"],
                ]
                table_data.append(row)

    # Print the classification report using tabulate
    headers = ["Class", "Precision", "Recall", "F1-Score", "Support"]
    m_table = tabulate(table_data, headers=headers, tablefmt="psql", floatfmt=".4f")
    print(m_table)


def eval_model(trainer, test_set, target_names, label):
    """
    Evaluate the model on the test set

    Args:

        trainer (transformers.Trainer): The trainer to use for the evaluation
        test_set (pandas.DataFrame): The test set to evaluate on
        target_names (list): The list of target names
        label (str): The label to use for the evaluation

    Returns:
        None
    """
    predictions = trainer.predict(test_set)
    predicted_labels = predictions.predictions.argmax(axis=1)
    ground_truth_labels = test_set[label]
    classification_reports = classification_report(
        ground_truth_labels,
        predicted_labels,
        target_names=target_names,
        output_dict=True,
    )
    print_using_tabulate(classification_reports)
    print("\n\n\n")


# metrics used to evaluate the model
load_accuracy = evaluate.load("accuracy")
load_f1 = evaluate.load("f1")


def compute_metrics(eval_pred):
    """
    Compute the metrics for the evaluation

    Args:

        eval_pred (tuple): The tuple containing the predictions and the labels

    Returns:
        dict: The dictionary containing the metrics
    """
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = load_accuracy.compute(predictions=predictions, references=labels)[
        "accuracy"
    ]
    f1 = load_f1.compute(
        predictions=predictions, references=labels, average="weighted"
    )["f1"]
    return {"accuracy": accuracy, "f1": f1}


def preprocess_function(examples):
    """
    Preprocess the data

    Args:

        examples (dict): The examples to preprocess

    Returns:
        dict: The preprocessed examples
    """

    inputs = tokenizer(examples["review_body"], truncation=True)
    inputs["labels"] = examples["stars"]
    return inputs


def train_model(
    model_name,
    num_of_labels,
    dataset,
    tokenizer,
    path_to_save,
    epochs=1,
    disable_tqdm=False,
    batch_size=8,
):
    """
    Train the model

    Args:

        model_name (str): The name of the model to use
        num_of_labels (int): The number of labels
        dataset (datasets.DatasetDict): The dataset to use
        tokenizer (transformers.tokenization_utils_base.PreTrainedTokenizerBase): The tokenizer to use
        path_to_save (str): The path to save the model to
        epochs (int): The number of epochs to train for
        disable_tqdm (bool): Whether to disable the tqdm progress bar
        batch_size (int): The batch size to use

    Returns:
        transformers.Trainer: The trainer for future use
    """

    training_args = TrainingArguments(
        output_dir=path_to_save,
        warmup_steps=10000,
        optim="adamw_torch",
        num_train_epochs=epochs,
        weight_decay=1e-4,
        evaluation_strategy=IntervalStrategy.EPOCH,
        save_strategy=IntervalStrategy.EPOCH,
        metric_for_best_model="f1",
        save_total_limit=1,
        disable_tqdm=disable_tqdm,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
    )

    model = AutoModelForSequenceClassification.from_pretrained(
        model_name, num_labels=num_of_labels
    )
    tokenized_train = dataset["train"].map(preprocess_function, batched=True)
    tokenized_validation = dataset["validation"].map(preprocess_function, batched=True)
    data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_train,
        eval_dataset=tokenized_validation,
        tokenizer=tokenizer,
        data_collator=data_collator,
        compute_metrics=compute_metrics,
    )

    return trainer


def prepare_dataset(train_df, test_df, valid_df, translated_df):
    """
    Prepare the dataset for training

    Args:

        train_df (pandas.DataFrame): The training set
        test_df (pandas.DataFrame): The test set
        valid_df (pandas.DataFrame): The validation set
        translated_df (pandas.DataFrame): The translated set

    Returns:
        datasets.DatasetDict: The dataset to use for training
    """

    train_ds = Dataset.from_pandas(train_df)
    test_ds = Dataset.from_pandas(test_df)
    valid_ds = Dataset.from_pandas(valid_df)
    translated_ds = Dataset.from_pandas(translated_df)

    dataset = DatasetDict()

    dataset["train"] = train_ds
    dataset["validation"] = valid_ds
    dataset["test"] = test_ds
    dataset["translated"] = translated_ds

    return dataset


def run_and_eval(
    train_df,
    test_df,
    valid_df,
    stars,
    languages,
    columns_to_drop,
    rows_to_drop,
    label,
    mapping,
    model_name,
    num_of_labels,
    M,
    N,
    label_names,
    verbose=False,
    epochs=1,
    disable_tqdm=True,
    batch_size=8,
):
    """
    Run the model and evaluate it

    Args:

        train_df (pandas.DataFrame): The training set
        test_df (pandas.DataFrame): The test set
        valid_df (pandas.DataFrame): The validation set
        stars (list): The stars to use
        languages (list): The languages to use
        columns_to_drop (list): The columns to drop
        rows_to_drop (list): The rows to drop
        label (str): The label to use
        mapping (dict): The mapping to use
        model_name (str): The name of the model to use
        num_of_labels (int): The number of labels
        M (int): Number of tail tokens to truncate
        N (int): Number of head tokens to truncate
        label_names (list): The names of the labels
        verbose (bool): Whether to print the results
        epochs (int): The number of epochs to train for
        disable_tqdm (bool): Whether to disable the tqdm progress bar
        batch_size (int): The batch size to use

    Returns:
        None
    """

    train, test, val = prepare_data(
        train_df, test_df, valid_df, columns_to_drop, stars, languages
    )
    train = reduce_dataset(train, stars, languages, rows_to_drop, verbose)
    train = replace_mapping(train, label, mapping)
    test = replace_mapping(test, label, mapping)
    val = replace_mapping(val, label, mapping)

    translated = drop_data(translated_test_df, stars, ["de", "es", "fr"], [])
    translated = replace_mapping(translated, label, mapping)

    global tokenizer
    tokenizer = AutoTokenizer.from_pretrained(model_name, num_labels=num_of_labels)
    train = prepare_truncation(train, tokenizer, M, N)
    test = prepare_truncation(test, tokenizer, M, N)
    val = prepare_truncation(val, tokenizer, M, N)
    translated = prepare_truncation(translated, tokenizer, M, N)

    dataset = prepare_dataset(train, test, val, translated)
    trainer = train_model(
        model_name,
        num_of_labels,
        dataset,
        tokenizer,
        f"./{model_name}_labels_{num_of_labels}_M_{M}_N_{N}",
        epochs=epochs,
        disable_tqdm=disable_tqdm,
        batch_size=batch_size,
    )

    trainer.train()

    print(f"Results for model: {model_name}, M = {M}, N = {N}")
    print("English test set results")
    tokenized_train = dataset["test"].map(preprocess_function, batched=True)
    eval_model(trainer, tokenized_train, label_names, label)
    print()
    print("Translated to English set results")
    tokenized_train = dataset["translated"].map(preprocess_function, batched=True)
    eval_model(trainer, tokenized_train, label_names, label)
    del trainer
    torch.cuda.empty_cache()

In [13]:
columns_to_drop = ['review_id', 'product_id', 'reviewer_id', 'product_category']
tokenizer = None
rows_to_drop = 30000

We're reducing the number of rows to 10000 for each label.  
Then we're merging all the labels into two categories in the following manner:  
Labels 1, 2, and **3** become 0, and labels 4 and 5 become 1.  
Label 3 is neutral hence all parts that 3 is not omitted are called "with neutral".  
Before each subpart there is a short explanation about the truncation method - N - head tokens and M - tail tokens.

**BERT CASED**

In this part, we are fine-tuning the `bert-base-cased` model.  

**Two labels with Neutral; N=128, M=382; overall 510 tokens**

In [14]:
model_name = "bert-base-cased"
stars = [1, 2, 3, 4, 5]
languages = ["en"]
mapping = {1: 0, 2: 0, 3: 0, 4: 1, 5: 1}
num_of_labels = 2
label = "stars"
label_names = ["Negative", "Positive"]
epochs = 1

In [15]:
N = 128
M = 382


run_and_eval(
    train_df,
    test_df,
    valid_df,
    stars,
    languages,
    columns_to_drop,
    rows_to_drop,
    label,
    mapping,
    model_name,
    num_of_labels,
    M,
    N,
    label_names,
    False,
    epochs,
    False,
)

Processing reviews: 100%|██████████| 15000/15000 [00:03<00:00, 3839.81it/s]


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.345100,0.403006,0.850400,0.851868


Results for model: bert-base-cased, M = 382, N = 128
English test set results


+----------+-------------+----------+------------+-----------+
| Class    |   Precision |   Recall |   F1-Score |   Support |
|----------+-------------+----------+------------+-----------|
| Negative |      0.9329 |   0.8060 |     0.8648 |      3000 |
| Positive |      0.7583 |   0.9130 |     0.8285 |      2000 |
+----------+-------------+----------+------------+-----------+





Translated to English set results


+----------+-------------+----------+------------+-----------+
| Class    |   Precision |   Recall |   F1-Score |   Support |
|----------+-------------+----------+------------+-----------|
| Negative |      0.9399 |   0.7697 |     0.8463 |      9000 |
| Positive |      0.7283 |   0.9262 |     0.8154 |      6000 |
+----------+-------------+----------+------------+-----------+






**Two labels with Neutral; N=64, M=64; overall 128 tokens**

In [16]:
N = 64
M = 64

run_and_eval(
    train_df,
    test_df,
    valid_df,
    stars,
    languages,
    columns_to_drop,
    rows_to_drop,
    label,
    mapping,
    model_name,
    num_of_labels,
    M,
    N,
    label_names,
    False,
    epochs,
    False,
)

Processing reviews: 100%|██████████| 15000/15000 [00:04<00:00, 3549.95it/s]


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.359600,0.391421,0.869200,0.868661


Results for model: bert-base-cased, M = 64, N = 64
English test set results


+----------+-------------+----------+------------+-----------+
| Class    |   Precision |   Recall |   F1-Score |   Support |
|----------+-------------+----------+------------+-----------|
| Negative |      0.8780 |   0.9137 |     0.8955 |      3000 |
| Positive |      0.8621 |   0.8095 |     0.8350 |      2000 |
+----------+-------------+----------+------------+-----------+





Translated to English set results


+----------+-------------+----------+------------+-----------+
| Class    |   Precision |   Recall |   F1-Score |   Support |
|----------+-------------+----------+------------+-----------|
| Negative |      0.8776 |   0.8921 |     0.8848 |      9000 |
| Positive |      0.8340 |   0.8133 |     0.8236 |      6000 |
+----------+-------------+----------+------------+-----------+






**Two labels with Neutral; N=510, M=0; overall 510 tokens**

In [17]:
N = 510
M = 0

run_and_eval(
    train_df,
    test_df,
    valid_df,
    stars,
    languages,
    columns_to_drop,
    rows_to_drop,
    label,
    mapping,
    model_name,
    num_of_labels,
    M,
    N,
    label_names,
    False,
    epochs,
    False,
)

Processing reviews: 100%|██████████| 15000/15000 [00:04<00:00, 3701.19it/s]


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.352800,0.356827,0.873200,0.873334


Results for model: bert-base-cased, M = 0, N = 510
English test set results


+----------+-------------+----------+------------+-----------+
| Class    |   Precision |   Recall |   F1-Score |   Support |
|----------+-------------+----------+------------+-----------|
| Negative |      0.8884 |   0.8813 |     0.8849 |      3000 |
| Positive |      0.8241 |   0.8340 |     0.8290 |      2000 |
+----------+-------------+----------+------------+-----------+





Translated to English set results


+----------+-------------+----------+------------+-----------+
| Class    |   Precision |   Recall |   F1-Score |   Support |
|----------+-------------+----------+------------+-----------|
| Negative |      0.8940 |   0.8481 |     0.8705 |      9000 |
| Positive |      0.7885 |   0.8492 |     0.8177 |      6000 |
+----------+-------------+----------+------------+-----------+






**Two labels with Neutral; N=0, M=510; overall 510 tokens**

In [18]:
N = 0
M = 510

run_and_eval(
    train_df,
    test_df,
    valid_df,
    stars,
    languages,
    columns_to_drop,
    rows_to_drop,
    label,
    mapping,
    model_name,
    num_of_labels,
    M,
    N,
    label_names,
    False,
    epochs,
    False,
)

Processing reviews: 100%|██████████| 15000/15000 [00:04<00:00, 3704.32it/s]


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.347500,0.312681,0.879800,0.879164


Results for model: bert-base-cased, M = 510, N = 0
English test set results


+----------+-------------+----------+------------+-----------+
| Class    |   Precision |   Recall |   F1-Score |   Support |
|----------+-------------+----------+------------+-----------|
| Negative |      0.8825 |   0.9160 |     0.8989 |      3000 |
| Positive |      0.8664 |   0.8170 |     0.8410 |      2000 |
+----------+-------------+----------+------------+-----------+





Translated to English set results


+----------+-------------+----------+------------+-----------+
| Class    |   Precision |   Recall |   F1-Score |   Support |
|----------+-------------+----------+------------+-----------|
| Negative |      0.8884 |   0.8821 |     0.8853 |      9000 |
| Positive |      0.8250 |   0.8338 |     0.8294 |      6000 |
+----------+-------------+----------+------------+-----------+






In the following part we're experimenting with the XLNET model.

**XLNET 2 labels with Neutrals**

In [19]:
model_name = "xlnet-base-cased"

**Two labels with Neutral; N=128, M=382; overall 510 tokens**

In [20]:
N = 128
M = 382

run_and_eval(
    train_df,
    test_df,
    valid_df,
    stars,
    languages,
    columns_to_drop,
    rows_to_drop,
    label,
    mapping,
    model_name,
    num_of_labels,
    M,
    N,
    label_names,
    False,
    epochs,
    False,
)

Processing reviews: 100%|██████████| 15000/15000 [00:05<00:00, 2806.13it/s]


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.364100,0.293744,0.888400,0.888400


Results for model: xlnet-base-cased, M = 382, N = 128
English test set results


+----------+-------------+----------+------------+-----------+
| Class    |   Precision |   Recall |   F1-Score |   Support |
|----------+-------------+----------+------------+-----------|
| Negative |      0.8926 |   0.9083 |     0.9004 |      3000 |
| Positive |      0.8588 |   0.8360 |     0.8472 |      2000 |
+----------+-------------+----------+------------+-----------+





Translated to English set results


+----------+-------------+----------+------------+-----------+
| Class    |   Precision |   Recall |   F1-Score |   Support |
|----------+-------------+----------+------------+-----------|
| Negative |      0.9082 |   0.8628 |     0.8849 |      9000 |
| Positive |      0.8085 |   0.8692 |     0.8378 |      6000 |
+----------+-------------+----------+------------+-----------+






**Two labels with Neutral; N=64, M=64; overall 128 tokens**

In [21]:
N = 64
M = 64

run_and_eval(
    train_df,
    test_df,
    valid_df,
    stars,
    languages,
    columns_to_drop,
    rows_to_drop,
    label,
    mapping,
    model_name,
    num_of_labels,
    M,
    N,
    label_names,
    False,
    epochs,
    False,
)

Processing reviews: 100%|██████████| 15000/15000 [00:05<00:00, 2979.00it/s]


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.354600,0.433952,0.874400,0.875136


Results for model: xlnet-base-cased, M = 64, N = 64
English test set results


+----------+-------------+----------+------------+-----------+
| Class    |   Precision |   Recall |   F1-Score |   Support |
|----------+-------------+----------+------------+-----------|
| Negative |      0.9178 |   0.8670 |     0.8917 |      3000 |
| Positive |      0.8158 |   0.8835 |     0.8483 |      2000 |
+----------+-------------+----------+------------+-----------+





Translated to English set results


+----------+-------------+----------+------------+-----------+
| Class    |   Precision |   Recall |   F1-Score |   Support |
|----------+-------------+----------+------------+-----------|
| Negative |      0.9251 |   0.8344 |     0.8774 |      9000 |
| Positive |      0.7835 |   0.8987 |     0.8371 |      6000 |
+----------+-------------+----------+------------+-----------+






**Two labels with Neutral; N=510, M=0; overall 510 tokens**

In [22]:
N = 510
M = 0

run_and_eval(
    train_df,
    test_df,
    valid_df,
    stars,
    languages,
    columns_to_drop,
    rows_to_drop,
    label,
    mapping,
    model_name,
    num_of_labels,
    M,
    N,
    label_names,
    False,
    epochs,
    False,
)

Processing reviews: 100%|██████████| 15000/15000 [00:05<00:00, 2712.64it/s]


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.351300,0.367319,0.885200,0.885082


Results for model: xlnet-base-cased, M = 0, N = 510
English test set results


+----------+-------------+----------+------------+-----------+
| Class    |   Precision |   Recall |   F1-Score |   Support |
|----------+-------------+----------+------------+-----------|
| Negative |      0.8910 |   0.9043 |     0.8976 |      3000 |
| Positive |      0.8532 |   0.8340 |     0.8435 |      2000 |
+----------+-------------+----------+------------+-----------+





Translated to English set results


+----------+-------------+----------+------------+-----------+
| Class    |   Precision |   Recall |   F1-Score |   Support |
|----------+-------------+----------+------------+-----------|
| Negative |      0.8975 |   0.8720 |     0.8846 |      9000 |
| Positive |      0.8159 |   0.8507 |     0.8329 |      6000 |
+----------+-------------+----------+------------+-----------+




